In [1]:
import cv2
import numpy as np

In [2]:
def ImageStaking(images):
    rows = len(images)
    cols = len(images[0])
    for i in range(rows):
        for j in range(cols):
            images[i][j] = cv2.resize(images[i][j], (250, 250))
            if len(images[i][j].shape) == 2:
                images[i][j] = cv2.cvtColor(images[i][j], cv2.COLOR_GRAY2BGR)
    imgBlank = np.zeros((250, 250), np.uint32)
    hor = [imgBlank]*rows
    for i in range(rows):
        hor[i] = np.hstack(images[i])
    ver = np.vstack(hor)
    return ver

In [6]:
def empty(a):
    pass
img = cv2.imread("dustbin2.jpg")
cv2.namedWindow("Parameters")
cv2.resizeWindow("Parameters", (400, 300))
cv2.createTrackbar("Threshold1", "Parameters", 150, 255, empty)
cv2.createTrackbar("Threshold2", "Parameters", 255, 255, empty)


def getContours(img, imgcontours):
    contours, hierarchy = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if area > 30261:
            cv2.drawContours(imgcontours, contours, -1, (0, 255, 0), 3)
            peri = cv2.arcLength(cnt, True)
            approx = cv2.approxPolyDP(cnt, 0.02 * peri, True)
            x, y, w, h = cv2.boundingRect(approx)
            cv2.rectangle(imgcontours, (x, y), (x + w, y + h), (255, 0, 0), 3)
            cv2.putText(imgcontours, "Points: " + str(len(approx)), (x + w - 30, y + 20), cv2.FONT_HERSHEY_PLAIN, 1, (0, 0, 255), 2)

while True:
    imgcontours = img.copy()
    imgBlur = cv2.GaussianBlur(img, (5, 5), 1)
    imgGray = cv2.cvtColor(imgBlur, cv2.COLOR_BGR2GRAY)
    threshold1 = cv2.getTrackbarPos("Threshold1", "Parameters")
    threshold2 = cv2.getTrackbarPos("Threshold2", "Parameters")
    imgCanny = cv2.Canny(imgGray, threshold1, threshold2)
    kernal = np.ones((5, 5))
    imgDail = cv2.dilate(imgCanny, kernal, iterations=1)
    getContours(imgDail, imgcontours)
    staked = ImageStaking([[imgCanny, imgDail, imgcontours]])
    cv2.imshow("Result", staked)
    if cv2.waitKey(1) & 0XFF == ord("q"):
        break
cv2.destroyAllWindows()